In [1]:
# RUN IT ONCE TO DOWNLOAD THE TEST DATASET
# !wget https://raw.githubusercontent.com/ShishirPatil/gorilla/main/berkeley-function-call-leaderboard/data/possible_answer/gorilla_openfunctions_v1_test_simple.json

In [2]:
file_path = "gorilla_openfunctions_v1_test_simple.json"

In [3]:
# standardize possible_answers from gorilla simple test set
import json

possible_answers = []
# Open the file and read line by line
with open(file_path, 'r') as file:
    for line in file:
        # Parse the JSON line into a Python dictionary
        json_data = json.loads(line)
        
        # Process the JSON object
        for function_name, params in json_data.items():
            final_string = function_name + ',{'
            for param_name, param_value in params.items():
                final_string += '"' + param_name + '":' + str(param_value) +','
            final_string = final_string[:-1] +"}"
            final_string = final_string.replace("'", '"')
            final_string = final_string.replace(" ", "")
            possible_answers.append(final_string)

In [4]:
possible_answers[0]

'calculate_triangle_area,{"base":[10],"height":[5],"unit":["units",""]}'

In [5]:
# Download questions and functions from gorilla dataset
import pandas as pd
df = pd.read_json("https://huggingface.co/datasets/gorilla-llm/Berkeley-Function-Calling-Leaderboard/raw/main/gorilla_openfunctions_v1_test_simple.json", lines=True)
df.head()

,question,function
0,Find the area of a triangle with a base of 10 ...,"{'name': 'calculate_triangle_area', 'descripti..."
1,Calculate the factorial of 5 using math functi...,"{'name': 'math.factorial', 'description': 'Cal..."
2,Calculate the hypotenuse of a right triangle g...,"{'name': 'math.hypot', 'description': 'Calcula..."
3,Find the roots of a quadratic equation with co...,"{'name': 'algebra.quadratic_roots', 'descripti..."
4,"Solve a quadratic equation where a=2, b=6, and...","{'name': 'solve_quadratic_equation', 'descript..."


In [6]:
from llama_cpp import Llama

llm = Llama("/home/asilva/models/Hermes-2-Pro-Llama-3-8B-F16.gguf", n_gpu_layers=-1)

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /home/asilva/models/Hermes-2-Pro-Llama-3-8B-F16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = Hermes-2-Pro-Llama-3-8B
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              l

In [7]:
# extract from the response only the first of nested brackets
import regex

def extract_first_of_nested_brackets(input_string):
    # Pattern to match the first complete set of brackets including nested ones
    pattern = r'\{(?:[^\{\}]*|(?R))*\}'
    match = regex.search(pattern, input_string)
    if match:
        return match.group(0)  # Returns the matched text including brackets
    return None

## Construction of the prompt for Hermes 2 Pro

In [8]:
pos = 0

In [9]:
question = df.iloc[pos]["question"]
question

'Find the area of a triangle with a base of 10 units and height of 5 units.'

In [10]:
function = df.iloc[pos]['function']
json.dumps(function)

'{"name": "calculate_triangle_area", "description": "Calculate the area of a triangle given its base and height.", "parameters": {"type": "dict", "properties": {"base": {"type": "integer", "description": "The base of the triangle."}, "height": {"type": "integer", "description": "The height of the triangle."}, "unit": {"type": "string", "description": "The unit of measure (defaults to \'units\' if not specified)"}}, "required": ["base", "height"]}}'

In [11]:
import json_repair

In [12]:
final_prompt = (
    "<|im_start|>system\n"
    "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. "
    "You may call one or more functions to assist with the user query. "
    "Don't make assumptions about what values to plug into functions. "
    "Here are the available tools: <tools> [" + json.dumps(function) +
    "] </tools> Use the following pydantic model json schema for each tool call you will make: "
    "{'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']} "
    "For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows: "
    "<tool_call>\n"
    "{'arguments': <args-dict>, 'name': <function-name>}"
    "</tool_call><|im_end|>\n"
    "<|im_start|>user\n" + question +
    "<|im_start|>assistant\n"
    "<tool_call>"
)
final_prompt

'<|im_start|>system\nYou are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. You may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug into functions. Here are the available tools: <tools> [{"name": "calculate_triangle_area", "description": "Calculate the area of a triangle given its base and height.", "parameters": {"type": "dict", "properties": {"base": {"type": "integer", "description": "The base of the triangle."}, "height": {"type": "integer", "description": "The height of the triangle."}, "unit": {"type": "string", "description": "The unit of measure (defaults to \'units\' if not specified)"}}, "required": ["base", "height"]}}] </tools> Use the following pydantic model json schema for each tool call you will make: {\'title\': \'FunctionCall\', \'type\': \'object\', \'properties\': {\'arguments\': {\'title\': \'Arguments\', \'type\': \'object\'}, \'name\': {\'title\': 

In [13]:
response = llm(final_prompt, temperature=0, seed=42, max_tokens=None)


llama_print_timings:        load time =     247.09 ms
llama_print_timings:      sample time =      34.21 ms /    25 runs   (    1.37 ms per token,   730.82 tokens per second)
llama_print_timings: prompt eval time =     246.26 ms /   320 tokens (    0.77 ms per token,  1299.46 tokens per second)
llama_print_timings:        eval time =     995.10 ms /    24 runs   (   41.46 ms per token,    24.12 tokens per second)
llama_print_timings:       total time =    1291.82 ms /   344 tokens


In [14]:
response["choices"][0]["text"]

' {"arguments": {"base": 10, "height": 5}, "name": "calculate_triangle_area"}</tool_call>'

In [15]:
# extract from the response only the first of nested brackets
result = extract_first_of_nested_brackets(response['choices'][0]['text'])
result

'{"arguments": {"base": 10, "height": 5}, "name": "calculate_triangle_area"}'

In [16]:
# standardize model_output
final_result = str(json_repair.loads(result)["name"]) + "," + str(json_repair.loads(result)["arguments"])
final_result = final_result.replace("'",'"')
final_result = final_result.replace(" ", "")
final_result

'calculate_triangle_area,{"base":10,"height":5}'

## Run the simulation for all the TEST_CATEGORY = simple

In [17]:
model_outputs = []
for pos in range(len(df)):
    # Construction of the prompt for Hermes 2 Pro
    question = df.iloc[pos]["question"]
    function = df.iloc[pos]['function']
    final_prompt = (
        "<|im_start|>system\n"
        "You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags. "
        "You may call one or more functions to assist with the user query. "
        "Don't make assumptions about what values to plug into functions. "
        "Here are the available tools: <tools> [" + json.dumps(function) +
        "] </tools> Use the following pydantic model json schema for each tool call you will make: "
        "{'title': 'FunctionCall', 'type': 'object', 'properties': {'arguments': {'title': 'Arguments', 'type': 'object'}, 'name': {'title': 'Name', 'type': 'string'}}, 'required': ['arguments', 'name']} "
        "For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows: "
        "<tool_call>\n"
        "{'arguments': <args-dict>, 'name': <function-name>}"
        "</tool_call><|im_end|>\n"
        "<|im_start|>user\n" + question +
        "<|im_start|>assistant\n"
        "<tool_call>"
    )
    try:
        response = llm(final_prompt, temperature=0, seed=0, max_tokens=None)
        # extract from the response only the first of nested brackets
        result = extract_first_of_nested_brackets(response['choices'][0]['text'])
        # standardize model_output
        final_result = str(json_repair.loads(result)["name"]) + "," + str(json_repair.loads(result)["arguments"])
        final_result = final_result.replace("'",'"')
        final_result = final_result.replace(" ", "")
        model_outputs.append(final_result)
    except:
        model_outputs.append("Error")

Llama.generate: prefix-match hit

llama_print_timings:        load time =     247.09 ms
llama_print_timings:      sample time =      34.01 ms /    25 runs   (    1.36 ms per token,   735.16 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     0 tokens (    -nan ms per token,     -nan tokens per second)
llama_print_timings:        eval time =    1042.69 ms /    25 runs   (   41.71 ms per token,    23.98 tokens per second)
llama_print_timings:       total time =    1091.00 ms /    25 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     247.09 ms
llama_print_timings:      sample time =      27.59 ms /    20 runs   (    1.38 ms per token,   724.95 tokens per second)
llama_print_timings: prompt eval time =      90.99 ms /   195 tokens (    0.47 ms per token,  2143.00 tokens per second)
llama_print_timings:        eval time =     790.93 ms /    19 runs   (   41.63 ms per token,    24.02 tokens per second)
llama_print_timings:       to

In [18]:
df = pd.DataFrame()
df["possible_answer"] = possible_answers
df["model_output"] = model_outputs
df.head()

,possible_answer,model_output
0,"calculate_triangle_area,{""base"":[10],""height"":...","calculate_triangle_area,{""base"":10,""height"":5}"
1,"math.factorial,{""number"":[5]}","math.factorial,{""number"":5}"
2,"math.hypot,{""x"":[4],""y"":[5],""z"":["""",0]}","math.hypot,{""x"":4,""y"":5}"
3,"algebra.quadratic_roots,{""a"":[1],""b"":[-3],""c"":...","algebra.quadratic_roots,{""a"":1,""b"":-3,""c"":2}"
4,"solve_quadratic_equation,{""a"":[2],""b"":[6],""c"":...","solve_quadratic_equation,{""a"":2,""b"":6,""c"":5}"


In [19]:
df.to_csv("results_Hermes2Pro.csv", index=False)